# Devlog 2023-07-12

Re-creating the maricopa_cbg_2019 GEO as part of a data reconciliation effort.


In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import pygris
from census import Census

from epymorph.data_shape import Shapes
from epymorph.error import GeoValidationException
from epymorph.geo.spec import (LABEL, AttribDef, CentroidDType, StaticGeoSpec,
                               Year)
from epymorph.geo.static import StaticGeo
from epymorph.geo.static import StaticGeoFileOps as F

YEAR = 2019
NUM_COUNTIES = 2494

spec = StaticGeoSpec(
    attributes=[
        LABEL,
        AttribDef('geoid', np.str_, Shapes.N),
        AttribDef('centroid', CentroidDType, Shapes.N),
        AttribDef('population', np.int64, Shapes.N),
        AttribDef('population_by_age', np.int64, Shapes.NxA(3)),
        AttribDef('population_by_age_x6', np.int64, Shapes.NxA(6)),
        AttribDef('median_age', np.float64, Shapes.N),
        AttribDef('median_income', np.int64, Shapes.N),
        AttribDef('average_household_size', np.float64, Shapes.N),
        AttribDef('pop_density_km2', np.float64, Shapes.N),
        AttribDef('tract_gini_index', np.float64, Shapes.N),
        AttribDef('tract_median_income', np.int64, Shapes.N),
    ],
    time_period=Year(YEAR))

AGE_VARS = [
    "B01001_003E",  # Population (Male) 0-4 years
    "B01001_004E",  # Population (Male) 5-9 years
    "B01001_005E",  # Population (Male) 10-14 years
    "B01001_006E",  # Population (Male) 15-17 years
    "B01001_007E",  # Population (Male) 18-19 years
    "B01001_008E",  # Population (Male) 20 years
    "B01001_009E",  # Population (Male) 21 years
    "B01001_010E",  # Population (Male) 22-24 years
    "B01001_011E",  # Population (Male) 25-29 years
    "B01001_012E",  # Population (Male) 30-34 years
    "B01001_013E",  # Population (Male) 35-39 years
    "B01001_014E",  # Population (Male) 40-44 years
    "B01001_015E",  # Population (Male) 45-49 years
    "B01001_016E",  # Population (Male) 50-54 years
    "B01001_017E",  # Population (Male) 55-59 years
    "B01001_018E",  # Population (Male) 60-61 years
    "B01001_019E",  # Population (Male) 62-64 years
    "B01001_020E",  # Population (Male) 65-66 years
    "B01001_021E",  # Population (Male) 67-69 years
    "B01001_022E",  # Population (Male) 70-74 years
    "B01001_023E",  # Population (Male) 75-79 years
    "B01001_024E",  # Population (Male) 80-84 years
    "B01001_025E",  # Population (Male) 85+ years
    "B01001_027E",  # Population (Female) 0-4 years
    "B01001_028E",  # Population (Female) 5-9 years
    "B01001_029E",  # Population (Female) 10-14 years
    "B01001_030E",  # Population (Female) 15-17 years
    "B01001_031E",  # Population (Female) 18-19 years
    "B01001_032E",  # Population (Female) 20 years
    "B01001_033E",  # Population (Female) 21 years
    "B01001_034E",  # Population (Female) 22-24 years
    "B01001_035E",  # Population (Female) 25-29 years
    "B01001_036E",  # Population (Female) 30-34 years
    "B01001_037E",  # Population (Female) 35-39 years
    "B01001_038E",  # Population (Female) 40-44 years
    "B01001_039E",  # Population (Female) 45-49 years
    "B01001_040E",  # Population (Female) 50-54 years
    "B01001_041E",  # Population (Female) 55-59 years
    "B01001_042E",  # Population (Female) 60-61 years
    "B01001_043E",  # Population (Female) 62-64 years
    "B01001_044E",  # Population (Female) 65-66 years
    "B01001_045E",  # Population (Female) 67-69 years
    "B01001_046E",  # Population (Female) 70-74 years
    "B01001_047E",  # Population (Female) 75-79 years
    "B01001_048E",  # Population (Female) 80-84 years
    "B01001_049E"   # Population (Female) 85+ years
]

census = Census(os.environ['CENSUS_API_KEY'])

In [2]:
STATE_FIPS = '04'
COUNTY_FIPS = '013'

# CBG data
query_cbg = {'for': 'block group: *',
             'in': f'state: {STATE_FIPS} county: {COUNTY_FIPS}'}

cbgs_raw = census.acs5.get([
    "B01003_001E",  # Total Population
    "B01002_001E",  # Median Age
    "B19013_001E",  # Median Household Income, infl.adj. Dollars, Past 12 Months
    "B19025_001E",  # Aggregate Household Income, infl.adj. Dollars, Past 12 Months
    "B25010_001E"   # Average Household Size
], query_cbg, year=YEAR)

cbgs_age_raw = census.acs5.get(AGE_VARS, query_cbg, year=YEAR)

cbgs_geog = pygris.block_groups(
    state=STATE_FIPS,
    county=COUNTY_FIPS,
    year=YEAR,
    cache=True
)

# Tract data
query_tract = {'for': 'tract: *',
               'in': f'state: {STATE_FIPS} county: {COUNTY_FIPS}'}

tracts_raw = census.acs5.get([
    "B19013_001E",  # Median Household Income, infl.adj. Dollars, Past 12 Months
    "B19083_001E"   # Gini Index (0 equality -> 1 inequality)
], query_tract, year=YEAR)

In [3]:
tracts = pd.DataFrame.from_records(tracts_raw)

geoids = tracts['state'] + tracts['county'] + tracts['tract']

tracts = pd.DataFrame({
    'tract_geoid': geoids,
    'tract_median_income': tracts['B19013_001E'].astype(np.int64).fillna(0).replace(-666666666, 0),
    'tract_gini_index': tracts['B19083_001E'].astype(np.float64).fillna(0.5).replace(-666666666, 0.5)
})

tracts[0:4]

,tract_geoid,tract_median_income,tract_gini_index
0,04013040507,52659,0.4376
1,04013040506,47425,0.3515
2,04013061009,86030,0.4178
3,04013071504,45453,0.3528


In [4]:
cbgs = pd.DataFrame.from_records(cbgs_raw)
cbgs.fillna(0, inplace=True)
cbgs.replace(-666666666, 0, inplace=True)

tract_geoids = cbgs['state'] + cbgs['county'] + cbgs['tract']
geoids = tract_geoids + cbgs['block group']

cbgs = pd.DataFrame({
    'geoid': geoids,
    'tract_geoid': tract_geoids,
    'population': cbgs['B01003_001E'].astype(np.int64),
    'median_age': cbgs['B01002_001E'].astype(np.float64),
    'median_income': cbgs['B19013_001E'].astype(np.int64),
    'total_income': cbgs['B19025_001E'].astype(np.int64),
    'average_household_size': cbgs['B25010_001E'].astype(np.float64)
})

# Merge geo info
cbgs = cbgs.merge(pd.DataFrame({
    'geoid': cbgs_geog['GEOID'],
    'centroid': cbgs_geog['geometry'].apply(lambda row: row.centroid.coords[0]),
    # TIGER areas are in m^2; divide by 1e6 to get km^2
    'area': cbgs_geog.ALAND / 1e6
}), on='geoid')

cbgs.insert(9, 'pop_density_km2', cbgs['population'] / cbgs['area'])
cbgs.drop(columns=['area'], inplace=True)

# Merge tract info
cbgs = cbgs.merge(tracts, on='tract_geoid')
cbgs.drop(columns=['tract_geoid'], inplace=True)

# Filter CBGs
cbgs.drop(cbgs[
    (cbgs['median_income'] == 0) &
    (cbgs['tract_median_income'] == 0)
].index, inplace=True)

cbgs.sort_values(by='geoid', inplace=True)
cbgs.reset_index(drop=True, inplace=True)

cbgs[0:4]

,geoid,population,median_age,median_income,total_income,average_household_size,centroid,pop_density_km2,tract_median_income,tract_gini_index
0,040130101011,1791,50.5,99219,90125800,2.33,"(-111.77075643330394, 33.76924660709943)",99.319999,99489,0.4423
1,040130101012,2007,63.9,127614,155213500,2.15,"(-111.72000671148204, 33.7410933260103)",68.782398,99489,0.4423
2,040130101013,1264,70.8,80742,73694600,1.83,"(-111.66697569766907, 33.72404113568911)",183.890869,99489,0.4423
3,040130101021,1139,58.9,105987,89511200,2.04,"(-111.49136471795472, 33.71546544071483)",0.480805,134865,0.5439


In [5]:
# We can define age brackets by how many columns are in each age group (per sex).
# Assumes the male and female groups have corresponding columns (which they do).

cbgs_age = pd.DataFrame.from_records(cbgs_age_raw)
cbgs_age.astype(np.int64)

geoids = cbgs_age['state'] + cbgs_age['county'] + \
    cbgs_age['tract'] + cbgs_age['block group']
cbgs_age.insert(len(cbgs_age.columns), 'geoid', geoids)

# Drop the same CBGs as above using an inner merge
cbgs_age = cbgs[['geoid']].merge(cbgs_age, how='inner', on='geoid')
cbgs_age.reset_index(drop=True, inplace=True)
cbgs_age.drop(columns=['geoid'], inplace=True)


def bracketize(brackets: dict[str, int]) -> pd.DataFrame:
    """Using the brackets info sum the age groups for both male and female."""
    sex_offset = sum(brackets.values())

    ranges = dict[str, tuple[slice, slice]]()
    age_offset = 0
    for key, count in brackets.items():
        i_m = age_offset
        i_f = age_offset + sex_offset
        ranges[key] = slice(i_m, i_m + count), slice(i_f, i_f + count)
        age_offset += count

    return pd.DataFrame({
        key: (cbgs_age.iloc[:, slice_m].sum(axis=1) +
              cbgs_age.iloc[:, slice_f].sum(axis=1))
        for key, (slice_m, slice_f) in ranges.items()
    }, dtype=np.int64)


cbgs_age_1 = bracketize({
    '00-19': 5,
    '20-64': 12,
    '65+': 6
})

cbgs_age_2 = bracketize({
    '00-19': 5,
    '20-34': 5,
    '35-54': 4,
    '55-64': 3,
    '65-74': 3,
    '75+': 3
})

cbgs_age_2[0:4]

,00-19,20-34,35-54,55-64,65-74,75+
0,358,110,671,374,194,84
1,162,0,423,460,559,403
2,0,22,0,249,655,338
3,58,52,361,362,258,48


In [6]:
values = {
    'label': cbgs['geoid'].to_numpy(dtype=np.str_),
    'geoid': cbgs['geoid'].to_numpy(dtype=np.str_),
    'centroid': cbgs['centroid'].to_numpy(dtype=CentroidDType),
    'population': cbgs['population'].to_numpy(dtype=np.int64),
    'population_by_age': cbgs_age_1.to_numpy(dtype=np.int64),
    'population_by_age_x6': cbgs_age_2.to_numpy(dtype=np.int64),
    'median_age': cbgs['median_age'].to_numpy(dtype=np.float64),
    'median_income': cbgs['median_income'].to_numpy(dtype=np.int64),
    'average_household_size': cbgs['average_household_size'].to_numpy(dtype=np.float64),
    'pop_density_km2': cbgs['pop_density_km2'].to_numpy(dtype=np.float64),
    'tract_median_income': cbgs['tract_median_income'].to_numpy(dtype=np.int64),
    'tract_gini_index': cbgs['tract_gini_index'].to_numpy(dtype=np.float64)
}

num_counties = len(values['label'])
if num_counties != NUM_COUNTIES:
    print(f"WARNING! Unexpected number of counties ({num_counties})!")

In [7]:
try:
    geo = StaticGeo(spec, values)
    geo.validate()
    geo.save(
        Path('epymorph/data/geo') / F.to_archive_filename('maricopa_cbg_2019')
    )
except GeoValidationException as e:
    print(e.pretty())

## Integrity check

Let's quickly check that I'm producing roughly the same values as before.

Say I have the old geo's npz in `maricopa_cbg_2019_geo_old.npz`:

In [8]:
# NOTE: this code cell won't run without the old data...
# if you really want to try it, this git command will checkout that old file:
# > git show ca35209^:epymorph/data/geo/maricopa_cbg_2019_geo.npz > epymorph/data/geo/maricopa_cbg_2019_geo_old.npz
# just don't commit the "old" file back to git...

from functools import partial

d1 = values
# with np.load('epymorph/data/geo/maricopa_cbg_2019_geo.npz') as file:
#     d1 = dict(file)

with np.load('epymorph/data/geo/maricopa_cbg_2019_geo_old.npz') as file:
    d2 = dict(file)
    # do the same filtering on the old one
    # that we did on the new one
    filter = np.isin(d2['labels'], d1['label'])
    for key, value in d2.items():
        d2[key] = value[filter]


def diff(field, arr1, arr2, comp) -> None:
    if len(arr1) != len(arr2):
        print(f"{field}: Lengths not equal: {len(arr1)} != {len(arr2)}")
        return

    diffs = False
    for i in range(len(arr1)):
        if not comp(arr1[i], arr2[i]):
            print(f"{field}: Difference at {i}: {arr1[i]} vs. {arr2[i]}")
            diffs = True

    if not diffs:
        print(f"{field}: No diffs!")


diff('label', d1['label'], d2['labels'], str.__eq__)
diff('population', d1['population'], d2['population'], np.int_.__eq__)
diff('median_age', d1['median_age'], d2['median_age'], np.isclose)
# there's no equivalent in the old geo to our new 'population_by_age'
diff('population_by_age_x6', d1['population_by_age_x6'],
     d2['pop_by_age'], np.array_equal)
diff('median_income', d1['median_income'], d2['median_income'], np.int_.__eq__)
diff('average_household_size', d1['average_household_size'],
     d2['average_household_size'], np.isclose)
# there's no equivalent in the old geo to our new 'tract_median_income'
diff('tract_gini_index', d1['tract_gini_index'], d2['tract_gini_index'], np.isclose)

# centroid values are not perfectly close... not sure what's up here
# np.allclose doesn't work on tuples, so convert to array form
d1centroid = np.asarray(d1['centroid'].tolist(), dtype=np.float64)
diff('centroid', d1centroid, d2['centroid'], partial(np.allclose, rtol=5e-5))

# pop density values are (sometimes) not close!
# presumably because I'm now using the pygris-supplied value for ALAND
# rather than calculating a polygon area -- this should be more correct,
# since some CBGs might contain bodies of water
diff('pop_density_km2', d1['pop_density_km2'], d2['pop_density_km2'],
     partial(np.isclose, rtol=0.6))

label: No diffs!
population: No diffs!
median_age: No diffs!
population_by_age_x6: No diffs!
median_income: No diffs!
average_household_size: No diffs!
tract_gini_index: No diffs!
centroid: No diffs!
pop_density_km2: No diffs!
